In [1]:
import tensorflow as tf

In [2]:
# constant
x = tf.constant(value=5, shape = [2, 2], name = 'x')
neg_x = tf.negative(x)
m = tf.multiply(x, neg_x)

In [3]:
# session with context
with tf.Session() as sess:
    print(sess.run(x))
    print(sess.run(neg_x))
    print(sess.run(m))

[[5 5]
 [5 5]]
[[-5 -5]
 [-5 -5]]
[[-25 -25]
 [-25 -25]]


In [4]:
# session without context
sess = tf.Session()
print(sess.run(m))
sess.close()

[[-25 -25]
 [-25 -25]]


In [5]:
# interactive Session
sess = tf.InteractiveSession()
a = tf.constant(5)
print(a.eval())
a_sqr = tf.multiply(a, a)
print(a_sqr.eval())

sess.close()

5
25


In [6]:
# logging- terminal required for logs
with tf.Session(config = tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(a_sqr))

25


When declaring variable using tf.get_variable(), of same statement is run again,it generates ValueError stating that the variable with given name already exist. If in variable_scope, reuse is set to True, it generates ValueError on first execution stating that variable with given name does not exist.
Setting reuse to tf.AUTO_REUSE solves the problem for now.

In [12]:
# variables
var = tf.Variable(False, name='var')

# tf.AUTO_REUSE is needed if this cell is to be run many times
with tf.variable_scope('test', reuse=tf.AUTO_REUSE):
    var1 = tf.get_variable(name='var1', shape=[3, 2]) 
var2 = tf.Variable([10.0, 1, 2], name='var2')

# return an operator that will initialise all variables
init_op = tf.global_variables_initializer()

# intialising variable
with tf.Session() as sess:
    # initialize variable var
    sess.run(var.initializer)
    # initialize all variable
    sess.run(init_op)
    
    print(sess.run(var))
    print(sess.run(var1))
    print(sess.run(var2))

var3 = tf.Variable(var2.initialized_value(), name='var3')
var4 = tf.Variable(var2.initialized_value() + var3.initialized_value(), name='var4')

# need to set reuse variable if we are getting it be get_variable
with tf.variable_scope('test', reuse=tf.AUTO_REUSE):
    var5 = tf.get_variable('var5', initializer=var2.initialized_value() * var3.initialized_value())

with tf.Session() as sess:
    # intialise dependent variable var3 with value of var2
    sess.run(var3.initializer)
    print(sess.run(var3))
    
    # initialize var4 with var2 + var3
    sess.run(var4.initializer)
    print(sess.run(var4))
    
    sess.run(var5.initializer)
    print(sess.run(var5))

False
[[ 0.7303157  -0.89372206]
 [-0.67146236 -0.3201579 ]
 [ 0.6619593   1.00053   ]]
[10.  1.  2.]
[10.  1.  2.]
[20.  2.  4.]
[100.   1.   4.]


In [2]:
# saving variables
sess = tf.InteractiveSession()

with tf.variable_scope('saving', reuse=tf.AUTO_REUSE):
    data = [1., 2., 8., -1., 0., 5.5, 6., 13]
    var = tf.get_variable(name='var', shape=[len(data)], dtype=tf.bool)

var.initializer.run()
print(var.eval())

# do something with var
for i in range(len(data)):
    if data[i] - data[i-1] > 0.5:
        value = var.eval()
        value[i] = True
        # assign it to var
        tf.assign(var, value).eval()
    print(var.eval()[i])

# save variable in a file
saver = tf.train.Saver()
path = saver.save(sess, './model.ckpt')
print('Saved to test-model to %s' %path)

sess.close()

[False False False False False False False False]
False
True
True
False
True
True
False
True
Saved to test-model to ./model.ckpt


In [3]:
sess = tf.InteractiveSession()

# create a variable with same name that you need to restore
with tf.variable_scope('saving', reuse=tf.AUTO_REUSE):
    var = tf.get_variable(name='var', shape=[8], dtype=tf.bool)

# load variable
saver = tf.train.Saver()

# restore variable
saver.restore(sess, './model.ckpt')
print(var.eval())
    
sess.close()

INFO:tensorflow:Restoring parameters from ./model.ckpt
[False  True  True False  True  True False  True]


In [10]:
# placeholders
sess = tf.InteractiveSession()
ph = tf.placeholder(dtype=tf.int32, shape = [10, 10], name='holder')
multiply = tf.matmul(ph, ph)

with tf.variable_scope('test', reuse=tf.AUTO_REUSE):
    value = tf.get_variable('value_var', shape=[10, 10], initializer=tf.constant_initializer(3), dtype = tf.int32)
    
value.initializer.run()
print(value.eval())
# assign 2 to all cells
assign = tf.assign(value, [[4] * 10] * 10)

assign.op.run()

v = value.eval()
print(v)
print(sess.run(multiply, feed_dict = {ph: v}))

sess.close()

[[3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]]
[[4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]]
[[160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]
 [160 160 160 160 160 160 160 160 160 160]]


Check visualisation using tensorboard --logdir=path/to/log

In [11]:
# TensorBoard
import numpy as np
data = np.random.normal(10, 1, 100)

alpha = tf.constant(0.5)
curr_value = tf.placeholder(dtype=tf.float32)
prev_avg = tf.Variable(0.)

update_avg = alpha * curr_value + (1 - alpha) * prev_avg

avg_hist = tf.summary.scalar('running_avg', update_avg)
value_hist = tf.summary.scalar('value', curr_value)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs')

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(data)):
        # run summary and moving avg tensors
        summary, curr_avg = sess.run([merged, update_avg], feed_dict={curr_value: data[i]})
        print(data[i], curr_avg)
        writer.add_summary(summary, i)

writer.close()

9.72586248229845 4.8629313
10.553835424623134 5.276918
9.442532369340734 4.7212663
10.876583135038768 5.4382915
9.465022960617866 4.7325115
10.753270220594972 5.376635
10.452520549913862 5.22626
9.599805408559472 4.799903
9.837089078931346 4.918545
9.420005588126873 4.710003
9.824842184914527 4.912421
9.659237984535505 4.829619
11.889078926346002 5.9445395
11.004504536104703 5.502252
10.659579397459932 5.3297896
10.276704065076188 5.138352
11.99159972801529 5.9958
11.26745830957762 5.633729
9.676616544584386 4.8383083
10.99413499209679 5.4970675
9.56234920448082 4.7811747
9.857299563245501 4.92865
10.21815517129961 5.1090775
10.766798379794059 5.383399
10.777616549925355 5.3888083
13.461752635247525 6.7308764
8.754079448936587 4.37704
9.372450476161967 4.6862254
10.4236387548239 5.211819
8.468372967315057 4.2341866
9.616619912250771 4.80831
8.542942182040228 4.271471
10.018093716221754 5.009047
10.134738028901259 5.067369
9.869362996654255 4.9346814
11.16838928598872 5.5841947
10.14154

Basic workflow of using QueueRunner and Coordinator
1. Create a queue.
2. Create enqueue, dequeue, training operations. Dequeue op is used in training op. A training op will be run in a session.
3. Create a coordinator. Pass this coordinator in QueueRunners' create_threads() and start the threads, either by setting start=True or calling start() explicilty.
4. Enqueue operations are also passed into create_threads() and start running in 3rd step.
5. In a session start your training op, which will use the dequeue op. Our queue is already being enqueued in multithreaded manner. The training process will consume data from our queue and use it in training the model.

In [14]:
# queue in tensorflow
import numpy as np
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count()

In [20]:
# create data here or the operation that will generate data
xs = np.random.randn(100, 3)
ys = np.random.randint(0, 2, size=100)

# create a queue
queue = tf.FIFOQueue(capacity=1000, dtypes=[xs.dtype, ys.dtype])

# create enqueue and dequeue op
enqueue = queue.enqueue_many([xs, ys])
xs_op, ys_op = queue.dequeue()

# create a QueueRunner
qr = tf.train.QueueRunner(queue, [enqueue] * NUM_THREADS)

# create a session and coordinator
sess = tf.InteractiveSession()

coord = tf.train.Coordinator()

# tf.train.QueueRunner.create_threads()
enqueue_threads = qr.create_threads(sess, coord = coord, start=True)

for i in range(100):
    if coord.should_stop():
        break;
    x,y = sess.run([xs_op, ys_op])
    print(x, y)

# request thread stop
coord.request_stop()
coord.join(enqueue_threads)

sess.close()

[-0.90955425 -0.47002415  1.23118123] 0
[ 0.85858162 -0.32164869  0.69940206] 1
[1.54339051 0.97316203 0.095515  ] 0
[-0.58551175  0.02619339  0.69512265] 1
[ 0.1401834  -1.87736226  0.62373634] 0
[0.59221719 0.38014235 1.07143524] 1
[-0.11387078  1.07947308  1.53373847] 0
[ 1.01969501 -1.24659399  1.08914298] 0
[ 1.12581002 -1.01812786 -0.11727193] 0
[ 0.68886543 -1.5200982  -0.94184346] 1
[-1.07374665  0.87594357 -0.39225591] 1
[-1.90789862  1.28533645  0.24535804] 1
[-1.41351532 -0.17783759 -0.10527184] 0
[-1.25640886 -1.30917671  0.01721235] 0
[-0.19689772  1.23988039  1.65540959] 1
[ 0.27210442 -1.57070722  0.47038234] 0
[ 0.86208254  0.59922935 -2.56956781] 0
[ 0.23732182  0.02646383 -0.82253002] 0
[-1.51229766 -0.69924896 -0.80103555] 0
[ 1.28598657 -0.61302904  0.15864497] 1
[ 0.18814511 -1.41921506  0.54691031] 1
[0.2486041  0.27481111 0.18494945] 0
[ 0.79280035  0.77560491 -1.15631237] 0
[-0.67808415 -1.08251554 -0.01118354] 0
[-0.1529407  -0.19698452 -0.03498209] 0
[-0.94162

Workflow
* Create a queue runner and queue using strin_input_producer(). Pass the filename to be read as 1D array of constant string tensors.
* Pass the queue to a reader.
* Read through the reader that will generate key and value tensors.
* Optional decoding if file is csv.
* Optional preprocessing of input.
* Create session and coordinator.
* Start queue runners.
* Evaluate key and value tensors to get input data.

In [23]:
# read data using queues
# create a queue using string_input_producer()
# a 1D string tensor is passed, here a constant string tensor is passed
queue = tf.train.string_input_producer(['big.txt'])

# attach queue with a reader
reader = tf.TextLineReader()

key, value = reader.read(queue)

# start a session to read data
sess = tf.InteractiveSession()
coord = tf.train.Coordinator()

# a queue runner is automatically added in graph when string_input_producer() is called
threads = tf.train.start_queue_runners(coord=coord)

for i in range(100):
    k, v = sess.run([key, value])
    print(k, v)

b'big.txt:1' b'The Project Gutenberg EBook of The Adventures of Sherlock Holmes'
b'big.txt:2' b'by Sir Arthur Conan Doyle'
b'big.txt:3' b'(#15 in our series by Sir Arthur Conan Doyle)'
b'big.txt:4' b''
b'big.txt:5' b'Copyright laws are changing all over the world. Be sure to check the'
b'big.txt:6' b'copyright laws for your country before downloading or redistributing'
b'big.txt:7' b'this or any other Project Gutenberg eBook.'
b'big.txt:8' b''
b'big.txt:9' b'This header should be the first thing seen when viewing this Project'
b'big.txt:10' b'Gutenberg file.  Please do not remove it.  Do not change or edit the'
b'big.txt:11' b'header without written permission.'
b'big.txt:12' b''
b'big.txt:13' b'Please read the "legal small print," and other information about the'
b'big.txt:14' b'eBook and Project Gutenberg at the bottom of this file.  Included is'
b'big.txt:15' b'important information about your specific rights and restrictions in'
b'big.txt:16' b'how the file may be used.  You can al